## Estatísticas Anuais - CID-10 de auxílios por incapacidade temporária "acidentários" (quando relacionado a atividade profissional) 

Arquivos (Acidentários) baixados de: https://www.gov.br/trabalho-e-previdencia/pt-br/assuntos/previdencia-social/saude-e-seguranca-do-trabalhador/dados-de-acidentes-do-trabalho/tabelas-cid-10

### Link dos datasets utilizados e gerados por este notebook

https://drive.google.com/drive/folders/1ZvH_MjFpXiVSVAV-RT_TxpaY5T9pFnWu?usp=sharing

Este notebook realiza algumas transformações e limpezas dos datasets obtidos do Ministério do Trabalho e Previdência.

Ao fim dos processos, pode-se obter subconjuntos de dados mais limpos, podendo fazer a junção por colunas com totais anuais, filtrar as categorias de interesse, bem como gerar subsets por seleção de termos específicos em determinadas linhas.

Devido ao modelo dos datasets disponibilizados pelo referido órgão, os processos abaixo só se aplicam aos dados de 2017 em diante. Dados anteriores a 2017 estão disponíveis no site acima em formato pdf.

In [1]:
# Imports
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Funções

# Função ETL

def etl_data(df):
    '''Executa algumas transformações no dataset original e o salva como csv'''    
        
    # Exclui até linha de índice 12 e altera linha de índice 13 como cabeçalho.
    df.drop(df.index[0:13], inplace=True)
    df = df.rename(columns = df.iloc[0]).drop(df.index[0])    
    
    # Exclui as últimas 5 linhas
    df.drop(df.index[2070:], inplace=True)    
    
    # Converte para float64 e caracteres '-' são considerandos NaN
    colunas = ['Janeiro', 'Fevereiro', 'Março', 'Abril', 'Maio', 'Junho', 'Julho','Agosto', 'Setembro', 'Outubro', 'Novembro', 'Dezembro']
    df[colunas] = df[colunas].apply(pd.to_numeric, errors='coerce')    

    # Substitui Nan por 0
    df.fillna(0, inplace = True)    
    
    # Cria a coluna total do ano
    df['Total_' + str(i)] = df.sum(axis=1)    
    
    # set_index. Coluna 'CID10 CATEGORIA' como index
    #df = df.set_index('CID10 CATEGORIA')       
    
    # Salva o dataframe com as alterações no diretório /datasets_tratados
    #df.to_csv('datasets_tratados/' + str(i) + '.csv', sep=';', encoding='utf-8')      
    
    # Sugestão para salvar apenas com os totais anuais
    df.to_csv('datasets_tratados/' + str(i) + '.csv', sep=';', encoding='utf-8', columns = ['CID10 CATEGORIA', 'Total_' + str(i)], index=False) 
    

###########################################################################################################################
###########################################################################################################################

# Use para selecionar categorias desejadas
def selecionaCategoria(character):
    '''Cria uma tupla com os códigos da categoria. Em character, digitar somente letras de A-Z (entre aspas)'''
    character = character.upper()
    list = []
    for i in range(100):
        if i < 10:
            list.append(str(character) + str('0') + str(i))
        else:
            list.append(str(character) + str(i))
    my_tuple = tuple(list)
    return my_tuple


In [3]:
# Carrega os datasets originais e aplica a função etl_data()
for i in range(2017, 2022):
    path = 'datasets_originais/' +str(i) + '-cid10.xlsx'
    df = pd.read_excel(path)
    etl_data(df)

In [4]:
# Substitui os caracteres carregados errados devido à codificação
for i in range(2017, 2022):
    v = 'datasets_tratados/' + str(i) + '.csv'
    df = pd.read_csv(v, sep=';', on_bad_lines='skip')
    df['CID10 CATEGORIA'] = df['CID10 CATEGORIA'].map(lambda x: x.replace('?E', 'oe'))
    df['CID10 CATEGORIA'] = df['CID10 CATEGORIA'].map(lambda x: x.replace('?O', 'ao'))
    df['CID10 CATEGORIA'] = df['CID10 CATEGORIA'].map(lambda x: x.replace('ç', 'c'))
    df['CID10 CATEGORIA'] = df['CID10 CATEGORIA'].map(lambda x: x.replace('ã', 'a'))
    df['CID10 CATEGORIA'] = df['CID10 CATEGORIA'].map(lambda x: x.replace('õ', 'o'))
        
    df.to_csv('datasets_tratados/' + str(i) + '.csv', sep=';', encoding='utf-8', columns = ['CID10 CATEGORIA', 'Total_' + str(i)], index=False)

In [5]:
# Confere o arquivo salvo
a = pd.read_csv('datasets_tratados/2021.csv', sep=';', on_bad_lines='skip')

In [6]:
a.head(10)

,CID10 CATEGORIA,Total_2021
0,Total,152999.0
1,Capítulo I: Algumas doencas Infecciosas e para...,1180.0
2,A00:Colera,2.0
3,A02:Outras Infeccoes por Salmonella,1.0
4,"A15:Tuberculose Respiratoria, com Confirmacao ...",284.0
5,"A16:Tuberculose das Vias Respiratorias, sem Co...",91.0
6,A17:Tuberculose do Sistema Nervoso,2.0
7,A18:Tuberculose de Outros Orgaos,24.0
8,A19:Tuberculose Miliar,2.0
9,A23:Brucelose,6.0


In [7]:
a.tail(20)

,CID10 CATEGORIA,Total_2021
742,Z53:Pessoas em Contato com Servicos de Saude p...,23.0
743,Z56:Problemas Relacionados com O Emprego e com...,16.0
744,Z57 - Exposicao ocupacional a fatores de risco,1.0
745,Z60:Problemas Relacionados com O Meio Social,1.0
746,Z63:Outros Problemas Relacionados com O Grupo ...,1.0
747,Z65:Problemas Relacionados com Outras Circunst...,17.0
748,Z73:Problemas Relacionados com a Organizacao d...,247.0
749,Z75:Problemas Relacionados com as Facilidades ...,1.0
750,Z76:Pessoas em Contato com Os Servicos de Saud...,1.0
751,Z93:Orificios Artificiais,4.0


## Leitura dos Dataframes e Armazenamento em Variáveis

In [8]:
# Cria uma lista com o path dos datasets tratados
lista_csv = []
for i in range(2017, 2022):
    u = 'datasets_tratados/' +str(i) + '.csv'    
    lista_csv.append(u)

In [9]:
lista_csv

['datasets_tratados/2017.csv',
 'datasets_tratados/2018.csv',
 'datasets_tratados/2019.csv',
 'datasets_tratados/2020.csv',
 'datasets_tratados/2021.csv']

In [10]:
# Cria dataframes para cada ano
df2017 = pd.read_csv(lista_csv[0], sep=';', on_bad_lines='skip')
df2018 = pd.read_csv(lista_csv[1], sep=';', on_bad_lines='skip')
df2019 = pd.read_csv(lista_csv[2], sep=';', on_bad_lines='skip')
df2020 = pd.read_csv(lista_csv[3], sep=';', on_bad_lines='skip')
df2021 = pd.read_csv(lista_csv[4], sep=';', on_bad_lines='skip')

In [11]:
print(df2017.shape)
print(df2018.shape)
print(df2019.shape)
print(df2020.shape)
print(df2021.shape)

(2070, 2)
(2070, 2)
(2070, 2)
(2070, 2)
(762, 2)


## Seleciona Categorias

In [12]:
# Seleciona as categorias desejadas
catF = selecionaCategoria('f')
catZ = selecionaCategoria('z')

# Fornece prints para criar novos subsets com as referidas categorias
cats = ['F', 'Z']
for ano in range(2017, 2022):
    for cat in cats:
        print(f"df{ano}_{cat} = df{ano}[df{ano}['CID10 CATEGORIA'].str.startswith(cat{cat.upper()})]")

df2017_F = df2017[df2017['CID10 CATEGORIA'].str.startswith(catF)]
df2017_Z = df2017[df2017['CID10 CATEGORIA'].str.startswith(catZ)]
df2018_F = df2018[df2018['CID10 CATEGORIA'].str.startswith(catF)]
df2018_Z = df2018[df2018['CID10 CATEGORIA'].str.startswith(catZ)]
df2019_F = df2019[df2019['CID10 CATEGORIA'].str.startswith(catF)]
df2019_Z = df2019[df2019['CID10 CATEGORIA'].str.startswith(catZ)]
df2020_F = df2020[df2020['CID10 CATEGORIA'].str.startswith(catF)]
df2020_Z = df2020[df2020['CID10 CATEGORIA'].str.startswith(catZ)]
df2021_F = df2021[df2021['CID10 CATEGORIA'].str.startswith(catF)]
df2021_Z = df2021[df2021['CID10 CATEGORIA'].str.startswith(catZ)]


In [13]:
# Copia e cola o resultado acima para gerar os novos subsets

df2017_F = df2017[df2017['CID10 CATEGORIA'].str.startswith(catF)]
df2017_Z = df2017[df2017['CID10 CATEGORIA'].str.startswith(catZ)]
df2018_F = df2018[df2018['CID10 CATEGORIA'].str.startswith(catF)]
df2018_Z = df2018[df2018['CID10 CATEGORIA'].str.startswith(catZ)]
df2019_F = df2019[df2019['CID10 CATEGORIA'].str.startswith(catF)]
df2019_Z = df2019[df2019['CID10 CATEGORIA'].str.startswith(catZ)]
df2020_F = df2020[df2020['CID10 CATEGORIA'].str.startswith(catF)]
df2020_Z = df2020[df2020['CID10 CATEGORIA'].str.startswith(catZ)]
df2021_F = df2021[df2021['CID10 CATEGORIA'].str.startswith(catF)]
df2021_Z = df2021[df2021['CID10 CATEGORIA'].str.startswith(catZ)]

In [14]:
# Escolhendo-se mais de duas categorias, usa-se o método concat() para unir as categorias por ano.
d1 = pd.concat([df2017_F, df2017_Z])
d2 = pd.concat([df2018_F, df2018_Z])
d3 = pd.concat([df2019_F, df2019_Z])
d4 = pd.concat([df2020_F, df2020_Z])
d5 = pd.concat([df2021_F, df2021_Z])
d2

,CID10 CATEGORIA,Total_2018
420,F00:Demencia na Doenca de Alzheimer (G30.-+),0
421,F01:Demencia Vascular,0
422,F02:Demencia em Outras Doencas Classificadas e...,1
423,F03:Demencia Nao Especificada,0
424,F04:Sindrome Amnesica Organica Nao Induzida Pe...,0
...,...,...
2062,Z95:Presenca de Implantes e Enxertos Cardiacos...,2
2063,Z96:Presenca de Outros Implantes Funcionais,0
2064,Z97:Presenca de Outros Dispositivos Proteticos,0
2065,Z98:Outros Estados Pos-Cirurgicos,72


## Merge para Unir os Subsets Criados

In [15]:
print(d1.shape)
print(d2.shape)
print(d3.shape)
print(d4.shape)
print(d5.shape)

(162, 2)
(162, 2)
(162, 2)
(162, 2)
(61, 2)


In [16]:
# Cuidado ao unir o d5 (ano de 2021), pois apresenta menos registros
from functools import reduce
data_frames = [d1, d2, d3, d4]

#df_merged = reduce(lambda  left,right: pd.merge(left,right, left_index=True, right_index=True, how='right'), data_frames)
    
df_merged = reduce(lambda  left,right: pd.merge(left,right, on=['CID10 CATEGORIA'],how='right'), data_frames)
df_merged

,CID10 CATEGORIA,Total_2017,Total_2018,Total_2019,Total_2020
0,F00:Demencia na Doenca de Alzheimer (G30.-+),0.0,0.0,1,0
1,F01:Demencia Vascular,0.0,0.0,0,0
2,F02:Demencia em Outras Doencas Classificadas e...,0.0,1.0,1,1
3,F03:Demencia Nao Especificada,0.0,0.0,0,0
4,F04:Sindrome Amnesica Organica Nao Induzida Pe...,1.0,0.0,0,0
...,...,...,...,...,...
157,Z95:Presenca de Implantes e Enxertos Cardiacos...,0.0,2.0,0,0
158,Z96:Presenca de Outros Implantes Funcionais,1.0,0.0,2,0
159,Z97:Presenca de Outros Dispositivos Proteticos,0.0,0.0,1,0
160,Z98:Outros Estados Pos-Cirurgicos,55.0,72.0,97,36


In [17]:
df_merged.isna().sum()

CID10 CATEGORIA     0
Total_2017         12
Total_2018         12
Total_2019          0
Total_2020          0
dtype: int64

In [18]:
# Selecionando linhas específicas
termos = ['Neuroticos', 'ansiosos', 'maquinas']
pattern = '|'.join(termos)

df_merged[df_merged['CID10 CATEGORIA'].str.contains(pattern, case = False, na = False)]

,CID10 CATEGORIA,Total_2017,Total_2018,Total_2019,Total_2020
34,F40:Transtornos Fobico-Ansiosos,91.0,115.0,82,29
35,F41:Outros Transtornos Ansiosos,2251.0,2660.0,3167,1417
40,F48:Outros Transtornos Neuroticos,9.0,11.0,8,5
161,Z99:Dependencia de Maquinas e Dispositivos Cap...,NaN,NaN,0,0
